# Rich Logging and printing

In [1]:
# | default_exp logger

In [2]:
# | export
# | hide
from rich.console import Console
from rich.theme import Theme
from loguru import logger
from datetime import datetime
from fastcore.basics import patch_to, ifnone
from rich.logging import RichHandler
from pathlib import Path

# from torch_snippets.ipython import is_in_notebook

from functools import wraps
import time

In [3]:
# | export
# | hide
def get_console(width=None):
    return Console(
        width=width,
        theme=Theme(
            {
                "repr.number": "bold cyan",
                "repr.string": "bold green",
                "logging.level.info": "dim yellow",
                "logging.level.warning": "dim red",
                "logging.level.exception": "bold red",
            }
        ),
    )


console = get_console()
print = console.print

In [4]:
print("The Number is 128")
print(
    {
        "a": 1,
        "b": [
            {1, 2, 3},
            "lskjdf",
        ],
        "c": 1,
        "d": [
            {1, 2, 3},
            "lskjdf",
        ],
        "ae": 1,
        "bf": [
            {1, 2, 3},
            "lskjdf",
        ],
        "ag": 1,
        "ba": [
            {1, 2, 3},
            "lskjdf",
        ],
    }
)

The Number is 128

{
    'a': 1,
    'b': [{1, 2, 3}, 'lskjdf'],
    'c': 1,
    'd': [{1, 2, 3}, 'lskjdf'],
    'ae': 1,
    'bf': [{1, 2, 3}, 'lskjdf'],
    'ag': 1,
    'ba': [{1, 2, 3}, 'lskjdf']
}

In [5]:
print(r"\x86")

\x86

In [6]:
# | export
# | hide
@patch_to(RichHandler)
def render(
    self,
    *,
    record,
    traceback,
    message_renderable: "ConsoleRenderable",
) -> "ConsoleRenderable":
    """patched the renderer to print function name as well"""
    path = Path(record.pathname).name
    level = self.get_level_text(record)
    time_format = None if self.formatter is None else self.formatter.datefmt
    log_time = datetime.fromtimestamp(record.created)

    log_renderable = self._log_render(
        self.console,
        [message_renderable] if not traceback else [message_renderable, traceback],
        log_time=log_time,
        time_format=time_format,
        level=level,
        path=path,
        line_no=f"{record.funcName}:{record.lineno}",
        link_path=record.pathname if self.enable_link_path else None,
    )
    return log_renderable


def reset_logger(level="INFO", width=120, silent=True):
    if level is not None:
        [logger.remove() for _ in range(100)]
        logger.configure(
            handlers=[
                {
                    "sink": RichHandler(
                        rich_tracebacks=True,
                        console=console,
                        tracebacks_show_locals=False,
                    ),
                    "format": "<level>{message}</level>",
                    "backtrace": True,
                    "level": level,
                }
            ],
        )
    if width is not None:
        for handler_id in logger._core.handlers:
            try:
                handler = logger._core.handlers[handler_id]
                handler._sink._handler.console = get_console(width=width)
            except:
                ...
    if not silent:
        logger.info(f"reset logger's console width to {width} and level to {level}!")


reset_logger_width = lambda width: reset_logger(width=width)

reset_logger(width=100, silent=True)
reset_logger(width=100, silent=True)

logger = logger

Debug = lambda x, depth=0: logger.opt(depth=depth + 1).log("DEBUG", x)
Info = lambda x, depth=0: logger.opt(depth=depth + 1).log("INFO", x)
Warn = lambda x, depth=0: logger.opt(depth=depth + 1).log("WARNING", x)
Excep = lambda x, depth=0: logger.opt(depth=depth + 1).log("ERROR", x)

In [7]:
a = {1: 1221, 2: 2342}
del a[1]
a

{2: 2342}

In [8]:
Debug("TESTING {1,2,3}")
Info("TESTING {1,2,3}")
Warn("TESTING {1,2,3}")
Excep("TESTING {1,2,3}")

[09/21/22 01:01:38] DEBUG    TESTING {1,2,3}                                               ]8;id=535448;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_65923/3592958886.py\3592958886.py]8;;\:]8;id=62831;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_65923/3592958886.py#<module>:1\<module>:1]8;;\

                    INFO     TESTING {1,2,3}                                               ]8;id=240155;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_65923/3592958886.py\3592958886.py]8;;\:]8;id=824819;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_65923/3592958886.py#<module>:2\<module>:2]8;;\

                    WARNING  TESTING {1,2,3}                                               ]8;id=86446;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_65923/3592958886.py\3592958886.py]8;;\:]8;id=279191;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_65923/3592958886.py#<module>:3\<module>:3]8;;\

                    ERROR    TESTING {1,2,3}                                               ]8;id=824167;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_65923/3592958886.py\3592958886.py]8;;\:]8;id=642149;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_65923/3592958886.py#<module>:4\<module>:4]8;;\

In [10]:
# | export
def enter_exit(func):
    """
    Logs the time taken to execute a function along with entry & exit time stamps
    """
    logger_ = logger.opt(depth=1)

    @wraps(func)
    def function_timer(*args, **kwargs):
        tic = time.time()
        logger_.log("DEBUG", f"Entered function `{func.__name__}`")
        o = func(*args, **kwargs)
        toc = time.time()
        logger_.log(
            "DEBUG", f"Exiting function `{func.__name__}` after {toc-tic:.3f} seconds"
        )
        return o

    return function_timer

In [11]:
@enter_exit
def add(x, y):
    print("sleeping...")
    time.sleep(2)
    return x + y


add(1, 23)

[09/21/22 01:01:41] DEBUG    Entered function `add`                                        ]8;id=339608;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_65923/4246827219.py\4246827219.py]8;;\:]8;id=81917;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_65923/4246827219.py#<module>:8\<module>:8]8;;\

sleeping...

[09/21/22 01:01:43] DEBUG    Exiting function `add` after 2.010 seconds                    ]8;id=148095;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_65923/4246827219.py\4246827219.py]8;;\:]8;id=631253;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_65923/4246827219.py#<module>:8\<module>:8]8;;\

24

In [12]:
def do():
    try:
        1 / 0
    except Exception as e:
        # console.print_exception(max_frames=20)
        logger.exception(e)


def do2():
    do()


do2()

                    ERROR    division by zero                                                    ]8;id=550042;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_65923/3806461673.py\3806461673.py]8;;\:]8;id=3283;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_65923/3806461673.py#do:6\do:6]8;;\
                             Traceback (most recent call last):                                                    
                                                                                                                   
                               File "/Users/yeshwanth.y/miniconda3/lib/python3.9/runpy.py", line                   
                             197, in _run_module_as_main                                                           
                                 return _run_code(code, main_globals, None,                                        
                                        |         |     -> {'__name__': '__main__', '__doc__':                     
                             'Entry point for launching an IPython kernel.\n\nThis is separate                     
                             from the ipykernel pack...                                                            
                                        |         -> <code object <module> at 0x10fc10df0, file                    
                             "/Users/yeshwanth.y/miniconda3/lib/python3.9/site-packages/ipykerne                   
                             l_launcher.py",...                                                                    
                                        -> <function _run_code at 0x10fc14310>                                     
                               File "/Users/yeshwanth.y/miniconda3/lib/python3.9/runpy.py", line                   
                             87, in _run_code                                                                      
                                 exec(code, run_globals)                                                           
                                      |     -> {'__name__': '__main__', '__doc__': 'Entry point                    
                             for launching an IPython kernel.\n\nThis is separate from the                         
                             ipykernel pack...                                                                     
                                      -> <code object <module> at 0x10fc10df0, file                                
                             "/Users/yeshwanth.y/miniconda3/lib/python3.9/site-packages/ipykerne                   
                             l_launcher.py",...                                                                    
                               File                                                                                
                             "/Users/yeshwanth.y/miniconda3/lib/python3.9/site-packages/ipykerne                   
                             l_launcher.py", line 16, in <module>                                                  
                                 app.launch_new_instance()                                                         
                                 |   -> <bound method Application.launch_instance of <class                        
                             'ipykernel.kernelapp.IPKernelApp'>>                                                   
                                 -> <module 'ipykernel.kernelapp' from                                             
                             '/Users/yeshwanth.y/miniconda3/lib/python3.9/site-packages/ipykerne                   
                             l/kernelapp.py'>                                                                      
                               File                                                                                
                             "/Users/yeshwanth.y/miniconda3/lib/pytho